# Organize ML projects with Scikit-Learn

While Machine Learning is powerful, people often overestimate it: apply machine learning to your project, and all your problems will be solved. In reality, it's not this simple. To be effective, one needs to organize the work very well. In this notebook, we will walkthrough practical aspects of a ML project. To look at the big picture, let's start with a checklist below. It should work reasonably well for most ML projects, but make sure to adapt it to your needs:

1. **Define the scope of work and objective**
    * How is your solution be used?
    * How should performance be measured? Are there any contraints?
    * How would the problem be solved manually?
    * List the available assumptions, and verify if possible.
    
    
2. **Get the data**
    * Document where you can get that data
    * Store data in a workspace you can easily access
    * Convert the data to a format you can easily manipulate
    * Check the overview (size, type, sample, description, statistics)
    * Data cleaning
    
    
3. **EDA & Data transformation**
    * Study each attribute and its characteristics (missing values, type of distribution, usefulness)
    * Visualize the data
    * Study the correlations between attributes
    * Feature selection, Feature Engineering, Feature scaling
    * Write functions for all data transformations
    
    
4. **Train models**
    * Automate as much as possible
    * Train promising models quickly using standard parameters. Measure and compare their performance
    * Analyze the errors the models make
    * Shortlist the top three of five most promising models, preferring models that make different types of errors.


5. **Fine-tunning**
    * Treat data transformation choices as hyperparameters, expecially when you are not sure about them (e.g., replace missing values with zeros or with the median value)
    * Unless there are very few hyperparameter value to explore, prefer random search over grid search.
    * Try ensemble methods
    * Test your final model on the test set to estimate the generalizaiton error. Don't tweak your model again, you would start overfitting the test set.

## Example: Articles categorization

### Objectives

Build a model to determine the categories of articles. 

### Get Data

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [10]:
bbc = pd.read_csv('https://raw.githubusercontent.com/dhminh1024/practice_datasets/master/bbc-text.csv')

In [11]:
bbc.sample(5)

,category,text
790,tech,text message record smashed uk mobile owners c...
2216,entertainment,top stars join us tsunami tv show brad pitt r...
1595,entertainment,no charges against tv s cosby us comedian bill...
688,sport,lions blow to world cup winners british and ir...
1169,politics,pakistani women must not hide hiding women a...


In [12]:
bbc.head(5)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [13]:
bbc['category'].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

In [14]:
bbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [15]:
# Your code here
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

bbc['label'] = encoder.fit_transform(bbc['category'])
bbc.drop(['category'], axis=1, inplace=True)
bbc

,text,label
0,tv future in the hands of viewers with home th...,4
1,worldcom boss left books alone former worldc...,0
2,tigers wary of farrell gamble leicester say ...,3
3,yeading face newcastle in fa cup premiership s...,3
4,ocean s twelve raids box office ocean s twelve...,1
...,...,...
2220,cars pull down us retail figures us retail sal...,0
2221,kilroy unveils immigration policy ex-chatshow ...,2
2222,rem announce new glasgow concert us band rem h...,1
2223,how political squabbles snowball it s become c...,2


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split

count_vect = CountVectorizer(analyzer = "word")
count_vec_X = count_vect.fit_transform(bbc["text"])

cvtrain_x,cvtest_x,cvtrain_y,cvtest_y = train_test_split(count_vec_X,bbc["label"],test_size = 0.2)

In [ ]:
def train_model(model_classifier, train_x,test_x, train_y,test_y):
    model_classifier.fit(train_x,train_y)
    
    prediction = model_classifier.predict(test_x)
    print(prediction)
    
    return metrics.accuracy_score(prediction, test_y)

accuracy = train_model(linear_model.LogisticRegression(),cvtrain_x,cvtest_x,cvtrain_y,cvtest_y)
print("Logistic regression,count Vectors: ", accuracy)

In [19]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(cvtrain_x,cvtrain_y)
prediction = model.predict(cvtest_x)
print(prediction)
print(model.score(cvtest_x,cvtest_y))

[4 0 4 0 4 3 3 3 2 2 3 0 3 2 4 1 1 3 3 1 2 2 2 0 1 1 1 3 1 2 3 1 0 0 4 3 3
 3 0 3 3 0 3 1 0 3 1 0 2 2 3 4 3 4 0 0 2 3 4 4 1 3 3 2 1 0 3 4 1 0 1 4 3 2
 2 2 3 2 4 4 2 4 0 0 1 2 4 1 2 3 0 3 0 0 4 3 1 4 2 2 1 0 0 4 2 0 1 1 0 3 1
 3 4 3 4 0 2 2 4 4 0 3 1 1 3 2 3 2 3 3 3 3 4 3 0 0 0 2 1 4 3 1 3 3 4 2 2 0
 1 1 0 1 4 1 3 2 3 4 2 0 4 2 4 0 1 4 1 2 3 2 4 2 3 3 1 1 1 1 4 3 1 4 4 3 0
 2 2 2 4 3 2 0 0 4 0 0 3 1 0 3 3 4 2 4 0 4 4 2 1 2 0 4 0 3 1 1 2 1 4 0 3 0
 2 4 3 0 0 3 0 3 3 2 1 3 1 3 1 4 1 4 2 0 3 0 0 3 0 0 3 3 3 0 2 2 0 3 4 2 0
 1 1 0 0 0 0 4 0 4 1 0 3 3 0 4 1 0 2 3 4 1 3 3 0 0 0 0 4 0 3 4 4 4 3 0 0 3
 3 0 3 3 4 3 0 4 4 4 0 3 4 1 2 1 0 2 4 3 4 3 2 3 1 3 4 0 4 4 0 1 0 3 4 3 4
 3 1 3 0 3 3 3 3 4 3 2 4 4 0 3 4 4 2 0 4 3 2 2 4 4 4 4 2 3 0 0 3 3 3 3 2 1
 2 3 2 1 3 2 0 4 2 3 3 1 3 0 2 2 2 2 0 1 4 2 0 1 1 4 0 1 1 1 2 0 0 3 3 4 1
 4 0 3 2 3 2 4 0 3 1 1 0 0 3 2 3 0 0 3 2 0 1 4 3 2 2 0 4 0 0 0 0 1 3 0 0 0
 2]
0.9820224719101124
